<a href="https://colab.research.google.com/github/ValentinaEmili/Sign_language/blob/main/DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import numpy as np
import os

In [6]:
js_100 = pd.read_json("/content/drive/MyDrive/NLP/WLASL100.json")
folder = "/content/drive/MyDrive/NLP/dataset/subset_100/"
original_folder = "/content/drive/MyDrive/NLP/dataset/"

training_folder = folder + "train/"
validation_folder = folder + "val/"
test_folder = folder + "test/"

training_video = training_folder + "video/"
validation_video = validation_folder + "video/"
test_video = test_folder + "video/"

training_images = training_folder + "images/"
validation_images = validation_folder + "images/"
test_images = test_folder + "images/"

os.makedirs(training_video, exist_ok=True)
os.makedirs(validation_video, exist_ok=True)
os.makedirs(test_video, exist_ok=True)

os.makedirs(training_images, exist_ok=True)
os.makedirs(validation_images, exist_ok=True)
os.makedirs(test_images, exist_ok=True)


In [21]:
video_ids = []
for video in os.listdir(original_folder + "train/video/"):
  video_ids.append(int(video[:-4].split("_")[1]))
for video in os.listdir(original_folder + "val/video/"):
  video_ids.append(int(video[:-4].split("_")[1]))
for video in os.listdir(original_folder + "test/video/"):
  video_ids.append(int(video[:-4].split("_")[1]))

new_id = max(video_ids)+1

In [22]:
def add_gaussian_noise(seq, mean=0, std=0.01):
  noise = np.random(mean, std, seq.shape)
  return seq + noise

def jitter_joints(seq, jitter_scale=0.02):
  return seq + np.random.uniform(-jitter_scale, jitter_scale, seq.shape)

def drop_frames(seq, drop_prob=0.1):
  keep_mask = np.random.rand(seq.shape[0]) > drop_prob
  if np.sum(keep_mask) < 1:
    keep_mask[np.random.randint(0, seq.shape[0])] = True
  return seq[keep_mask]